# Standard Type Library

Pandapower provides a standard type library that allows the comfortable managing of type data for lines and transformers.

The following examples are all based on lines, but can be equally used for the transformer database by specifying element="trafo" instead of element="line" in all standard type functions.

## Basic Pandapower Standard Types

Pandapower comes with generic standard types that you can use to build generic networks:

In [1]:
import pandapower as pp
net = pp.create_empty_network()
pp.available_std_types(net, element="line")

,alpha,c_nf_per_km,max_i_ka,q_mm2,r_ohm_per_km,type,x_ohm_per_km
NAYY 4x50 SE,0.00403,210.00,0.142,50,0.6420,cs,0.083
NAYY 4x120 SE,0.00403,264.00,0.242,120,0.2250,cs,0.080
NAYY 4x150 SE,0.00403,261.00,0.270,150,0.2080,cs,0.080
NA2XS2Y 1x95 RM/25 12/20 kV,0.00403,216.00,0.252,95,0.3130,cs,0.132
NA2XS2Y 1x185 RM/25 12/20 kV,0.00403,273.00,0.362,185,0.1610,cs,0.117
NA2XS2Y 1x240 RM/25 12/20 kV,0.00403,304.00,0.421,240,0.1220,cs,0.112
NA2XS2Y 1x95 RM/25 6/10 kV,0.00403,315.00,0.249,95,0.3130,cs,0.123
NA2XS2Y 1x185 RM/25 6/10 kV,0.00403,406.00,0.358,185,0.1610,cs,0.110
NA2XS2Y 1x240 RM/25 6/10 kV,0.00403,456.00,0.416,240,0.1220,cs,0.105
NA2XS2Y 1x150 RM/25 12/20 kV,0.00403,250.00,0.319,150,0.2060,cs,0.116


If you create a line with the create_line function, the element parameters are taken from this library:

In [2]:
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
lid = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="NAYY 4x50 SE", name="test_line")
net.line.loc[lid]

name               test_line
std_type        NAYY 4x50 SE
from_bus                   0
to_bus                     1
length_km                0.1
r_ohm_per_km           0.642
x_ohm_per_km           0.083
c_nf_per_km              210
g_us_per_km                0
max_i_ka               0.142
df                         1
parallel                   1
type                      cs
in_service              True
Name: 0, dtype: object

## Create Individual Standard Types

Additionally to the basic pandapower standard types, you can define individual standard types for you pandapower network.

To create a new standard type you need at least the following parameters:

**For Lines**:
- r_ohm_per_km
- x_ohm_per_km
- c_nf_per_km
- max_i_ka

**For Transformers:**
- sn_mva
- vn_hv_kv
- vn_lv_kv
- vk_percent
- vkr_percent
- i0_percent
- pfe_kw
- shift_degree

To define the parameters, create a dictionary with the style {"parameter1": parameter_value1, "parameter2": paramter_value2,...} and save the standard type to your network with the create_std_type function:

In [4]:
net = pp.create_empty_network()
test_type = {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "max_i_ka": 0.4, "type": "cs"}
pp.create_std_type(net, name="test_type", data=test_type, element="line")
pp.available_std_types(net, element="line")

,alpha,c_nf_per_km,max_i_ka,q_mm2,r_ohm_per_km,type,x_ohm_per_km
NAYY 4x50 SE,0.00403,210.00,0.142,50.0,0.6420,cs,0.083
NAYY 4x120 SE,0.00403,264.00,0.242,120.0,0.2250,cs,0.080
NAYY 4x150 SE,0.00403,261.00,0.270,150.0,0.2080,cs,0.080
NA2XS2Y 1x95 RM/25 12/20 kV,0.00403,216.00,0.252,95.0,0.3130,cs,0.132
NA2XS2Y 1x185 RM/25 12/20 kV,0.00403,273.00,0.362,185.0,0.1610,cs,0.117
NA2XS2Y 1x240 RM/25 12/20 kV,0.00403,304.00,0.421,240.0,0.1220,cs,0.112
NA2XS2Y 1x95 RM/25 6/10 kV,0.00403,315.00,0.249,95.0,0.3130,cs,0.123
NA2XS2Y 1x185 RM/25 6/10 kV,0.00403,406.00,0.358,185.0,0.1610,cs,0.110
NA2XS2Y 1x240 RM/25 6/10 kV,0.00403,456.00,0.416,240.0,0.1220,cs,0.105
NA2XS2Y 1x150 RM/25 12/20 kV,0.00403,250.00,0.319,150.0,0.2060,cs,0.116


As you can see the test_type is now available in the standard type library, and you can use it to create a line:

In [5]:
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
lid = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="test_type", name="test_line")
print(net.line.loc[lid])

name            test_line
std_type        test_type
from_bus                0
to_bus                  1
length_km             0.1
r_ohm_per_km         0.01
x_ohm_per_km         0.02
c_nf_per_km            10
g_us_per_km             0
max_i_ka              0.4
df                      1
parallel                1
type                   cs
in_service           True
Name: 0, dtype: object


## Local Standard Type Libraries

Standard tpyes are saved with the network if you use the pp.to_json function and are therefore still available if you load the network again.

If you have a set of standard types that you want to save locally and apply to multiple networks, you can create a set of standard type parameters as a dictionary in the style {"std_type1: {"r_ohm_per_km": r_ohm_per_km,...}, "std_type2": {"r_ohm_per_km": r_ohm_per_km,...},...} and add it to the network with the create_std_types function:

In [7]:
net = pp.create_empty_network()
linetypes = {"typ1": {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "max_i_ka": 0.4, "type": "cs"},
             "typ2": {"r_ohm_per_km": 0.015, "x_ohm_per_km": 0.01, "c_nf_per_km": 30, "max_i_ka": 0.3, "type": "cs"}}
pp.create_std_types(net, data=linetypes, element="line")
pp.available_std_types(net)

,alpha,c_nf_per_km,max_i_ka,q_mm2,r_ohm_per_km,type,x_ohm_per_km
NAYY 4x50 SE,0.00403,210.00,0.142,50.0,0.6420,cs,0.083
NAYY 4x120 SE,0.00403,264.00,0.242,120.0,0.2250,cs,0.080
NAYY 4x150 SE,0.00403,261.00,0.270,150.0,0.2080,cs,0.080
NA2XS2Y 1x95 RM/25 12/20 kV,0.00403,216.00,0.252,95.0,0.3130,cs,0.132
NA2XS2Y 1x185 RM/25 12/20 kV,0.00403,273.00,0.362,185.0,0.1610,cs,0.117
NA2XS2Y 1x240 RM/25 12/20 kV,0.00403,304.00,0.421,240.0,0.1220,cs,0.112
NA2XS2Y 1x95 RM/25 6/10 kV,0.00403,315.00,0.249,95.0,0.3130,cs,0.123
NA2XS2Y 1x185 RM/25 6/10 kV,0.00403,406.00,0.358,185.0,0.1610,cs,0.110
NA2XS2Y 1x240 RM/25 6/10 kV,0.00403,456.00,0.416,240.0,0.1220,cs,0.105
NA2XS2Y 1x150 RM/25 12/20 kV,0.00403,250.00,0.319,150.0,0.2060,cs,0.116


The linetypes dictionary can be easily saved in a local project folder (e.g. using pickle or jsonpickle) and reloaded once it is needed. 

Another possibility is to copy the standard type library of an existing network:

In [8]:
new_net = pp.create_empty_network()
pp.copy_std_types(from_net=net, to_net=new_net, element="line")
pp.available_std_types(new_net)

,alpha,c_nf_per_km,max_i_ka,q_mm2,r_ohm_per_km,type,x_ohm_per_km
NAYY 4x50 SE,0.00403,210.00,0.142,50.0,0.6420,cs,0.083
NAYY 4x120 SE,0.00403,264.00,0.242,120.0,0.2250,cs,0.080
NAYY 4x150 SE,0.00403,261.00,0.270,150.0,0.2080,cs,0.080
NA2XS2Y 1x95 RM/25 12/20 kV,0.00403,216.00,0.252,95.0,0.3130,cs,0.132
NA2XS2Y 1x185 RM/25 12/20 kV,0.00403,273.00,0.362,185.0,0.1610,cs,0.117
NA2XS2Y 1x240 RM/25 12/20 kV,0.00403,304.00,0.421,240.0,0.1220,cs,0.112
NA2XS2Y 1x95 RM/25 6/10 kV,0.00403,315.00,0.249,95.0,0.3130,cs,0.123
NA2XS2Y 1x185 RM/25 6/10 kV,0.00403,406.00,0.358,185.0,0.1610,cs,0.110
NA2XS2Y 1x240 RM/25 6/10 kV,0.00403,456.00,0.416,240.0,0.1220,cs,0.105
NA2XS2Y 1x150 RM/25 12/20 kV,0.00403,250.00,0.319,150.0,0.2060,cs,0.116


## Add Additional Parameters

You can define additional parameters in the standard type library and load them into pandapower. For example, if you want to run a reliability analysis for which you need a hazard rate for each line, you will want to add the hazard rate "h_per_km" for your individual standard type:

In [9]:
import pandapower as pp
net = pp.create_empty_network()
test_type = {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "max_i_ka": 0.4, "type": "cs", "h_per_km": 0.01}
pp.create_std_type(net, name="test_type", data=test_type, element="line")
pp.available_std_types(net, element="line")

,alpha,c_nf_per_km,h_per_km,max_i_ka,q_mm2,r_ohm_per_km,type,x_ohm_per_km
NAYY 4x50 SE,0.00403,210.00,NaN,0.142,50.0,0.6420,cs,0.083
NAYY 4x120 SE,0.00403,264.00,NaN,0.242,120.0,0.2250,cs,0.080
NAYY 4x150 SE,0.00403,261.00,NaN,0.270,150.0,0.2080,cs,0.080
NA2XS2Y 1x95 RM/25 12/20 kV,0.00403,216.00,NaN,0.252,95.0,0.3130,cs,0.132
NA2XS2Y 1x185 RM/25 12/20 kV,0.00403,273.00,NaN,0.362,185.0,0.1610,cs,0.117
NA2XS2Y 1x240 RM/25 12/20 kV,0.00403,304.00,NaN,0.421,240.0,0.1220,cs,0.112
NA2XS2Y 1x95 RM/25 6/10 kV,0.00403,315.00,NaN,0.249,95.0,0.3130,cs,0.123
NA2XS2Y 1x185 RM/25 6/10 kV,0.00403,406.00,NaN,0.358,185.0,0.1610,cs,0.110
NA2XS2Y 1x240 RM/25 6/10 kV,0.00403,456.00,NaN,0.416,240.0,0.1220,cs,0.105
NA2XS2Y 1x150 RM/25 12/20 kV,0.00403,250.00,NaN,0.319,150.0,0.2060,cs,0.116


You can see that the parameter "h_per_km" is stored in the standard type database for the new standard type.

We now create two lines, one of the new "test_type" and one with a standard type that does not have the parameter "h_per_km".

In [10]:
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
l1 = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="test_type", name="test_line1")
l2 = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="NAYY 4x50 SE", name="test_line2")
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,test_line1,test_type,0,1,0.1,0.010,0.020,10.0,0.0,0.400,1.0,1,cs,True
1,test_line2,NAYY 4x50 SE,0,1,0.1,0.642,0.083,210.0,0.0,0.142,1.0,1,cs,True


The parameter "h_per_km" ist not in the line table, since it is not a pandapower standard parameter. You can however load the parameter from the database with the parameter_from_std_type function:

In [11]:
pp.parameter_from_std_type(net, parameter="h_per_km")
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,h_per_km
0,test_line1,test_type,0,1,0.1,0.010,0.020,10.0,0.0,0.400,1.0,1,cs,True,0.01
1,test_line2,NAYY 4x50 SE,0,1,0.1,0.642,0.083,210.0,0.0,0.142,1.0,1,cs,True,None


Pandapower adds the parameter to the linetable where it is available. The second line does not have a value for h_per_km, since it is not specified in the standard type database. You can however define a fallback (fill) value for lines that do not have a standard type or that have a standard type for which the desired parameter is not available:

In [12]:
pp.parameter_from_std_type(net, parameter="h_per_km", fill=0.02)
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,h_per_km
0,test_line1,test_type,0,1,0.1,0.010,0.020,10.0,0.0,0.400,1.0,1,cs,True,0.01
1,test_line2,NAYY 4x50 SE,0,1,0.1,0.642,0.083,210.0,0.0,0.142,1.0,1,cs,True,0.02


## Change Standard Type

The change_std_type function allows changing the standard type of an element.

In [13]:
net = pp.create_empty_network()
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
l2 = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="NAYY 4x50 SE", name="test_line")
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,test_line,NAYY 4x50 SE,0,1,0.1,0.642,0.083,210.0,0.0,0.142,1.0,1,cs,True


In [14]:
pp.change_std_type(net, l2, "NAYY 4x150 SE", element="line")
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,test_line,NAYY 4x150 SE,0,1,0.1,0.208,0.08,261.0,0.0,0.27,1.0,1,cs,True


## Find Standard Type

It is possible to search for a standard type given a set of parameters:

In [15]:
pp.find_std_type_by_parameter(net, data={"r_ohm_per_km": 0.1188, "x_ohm_per_km": 0.39})

['243-AL1/39-ST1A 110.0']

You can also define a tolerance epsilon to find types with similar parameters:

In [16]:
pp.find_std_type_by_parameter(net, data={"r_ohm_per_km": 0.12, "x_ohm_per_km": 0.40}, epsilon=.05)

['184-AL1/30-ST1A 110.0', '243-AL1/39-ST1A 110.0', '305-AL1/39-ST1A 110.0']